In [80]:
!pip install transformers

In [81]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from keras.preprocessing.sequence import pad_sequences

import torch
import numpy as np

In [82]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [83]:
output_dir = '/content/drive/My Drive/BERT Fine-Tuning/model_save/'

In [84]:
# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [105]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 512

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [106]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [107]:
sentiment_dic = {
    0 : 'anger',
    1 : 'fear',
    2 : 'joy',
    3 : 'love',
    4 : 'neutral',
    5 : 'sadness',
    6 : 'surprise',
}

# ['anger', 'fear', 'joy', 'love', 'neutral', 'sadness', 'surprise']


In [142]:
logits = test_sentences(['최태성의한국사를들어시험게합격해서기쁘다'])

print(logits)
print(sentiment_dic[np.argmax(logits)])

# label_encoder.inverse_transform(sentiment)

[[-0.38248304 -1.7115409   3.759876   -0.33307812 -0.04192595 -0.45470053
  -0.72178966]]
joy


In [158]:
# a = input()

a = "짱무섭다 ㅋㅋㅋㅋㅋㅋㅋ"

# ㅋㅋㅋ가 여러개면 ㅋㅋ로
a = a.replace("[ㄱ-ㅊ]|[ㅌㅍ]|[ㅏ-ㅣ]", "")
a = a.replace("[ㅎ]", " ㅋ ")
a = a.replace("[ㅋ]{2,}", " ㅋㅋ ")
a = a.replace("[.]", "")
# a = a.replace("[_ㅜㅠㅡㅓㅏ]","")
a = a.replace("&lt;|&gt;", "")
a = a.replace("[^\w]|_", " ")
a = a.replace("[ ]{2,}", " ")
a = a.replace("[아]{2,}",  "아아 ")
a = a.replace("[으]{2,}", " 으으 ")
a = a.replace("[하]{2,}", " 하하 ")
a[:10]
# text

print(len(a))
print(a)

print(a[:512])

12
짱무섭다 ㅋㅋㅋㅋㅋㅋㅋ
짱무섭다 ㅋㅋㅋㅋㅋㅋㅋ


In [154]:
logits = test_sentences([a])

print(logits)
print(sentiment_dic[np.argmax(logits)])
print(sentiment_dic)

# label_encoder.inverse_transform(sentiment)

[[-0.40022784 -0.28155577  1.0168293   0.71919805 -0.09995987  0.22612944
  -0.68637025]]
joy
{0: 'anger', 1: 'fear', 2: 'joy', 3: 'love', 4: 'neutral', 5: 'sadness', 6: 'surprise'}
